# CSCI544 - Assignment 3

Name: Anushka Deshpande 

The following versions of each package are used:

Python 3.10.9 | packaged by conda-forge | (main, Feb  2 2023, 20:24:27) [Clang 14.0.6 ] \
Pandas 1.5.3 \
Numpy  1.24.2 \
Scikit-learn 1.2.1 \
NLTK  3.8.1 \
Tensorflow 2.11.0 \
Gensim  4.3.0

## Importing Required Libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.utils import shuffle
import tensorflow as tf

2023-03-01 11:11:37.670301: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/anushkadeshpande/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anushkadeshpande/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anushkadeshpande/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## 1. Dataset Generation

### Read Dataset

We read the dataset using pandas library.

In [4]:
url = "data.tsv"
sdf = pd.read_csv(url, sep="\t", on_bad_lines='skip')
sdf.head() # shows first 5 entries

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,1797882,R3I2DHQBR577SS,B001ANOOOE,2102612,The Naked Bee Vitmin C Moisturizing Sunscreen ...,Beauty,5,0.0,0.0,N,Y,Five Stars,"Love this, excellent sun block!!",2015-08-31
1,US,18381298,R1QNE9NQFJC2Y4,B0016J22EQ,106393691,"Alba Botanica Sunless Tanning Lotion, 4 Ounce",Beauty,5,0.0,0.0,N,Y,Thank you Alba Bontanica!,The great thing about this cream is that it do...,2015-08-31
2,US,19242472,R3LIDG2Q4LJBAO,B00HU6UQAG,375449471,"Elysee Infusion Skin Therapy Elixir, 2oz.",Beauty,5,0.0,0.0,N,Y,Five Stars,"Great Product, I'm 65 years old and this is al...",2015-08-31
3,US,19551372,R3KSZHPAEVPEAL,B002HWS7RM,255651889,"Diane D722 Color, Perm And Conditioner Process...",Beauty,5,0.0,0.0,N,Y,GOOD DEAL!,I use them as shower caps & conditioning caps....,2015-08-31
4,US,14802407,RAI2OIG50KZ43,B00SM99KWU,116158747,Biore UV Aqua Rich Watery Essence SPF50+/PA+++...,Beauty,5,0.0,0.0,N,Y,this soaks in quick and provides a nice base f...,This is my go-to daily sunblock. It leaves no ...,2015-08-31


Now, we filter for required columns in the dataset.

In [5]:
new_df = sdf[['star_rating', 'review_body']]
new_df.head()

,star_rating,review_body
0,5,"Love this, excellent sun block!!"
1,5,The great thing about this cream is that it do...
2,5,"Great Product, I'm 65 years old and this is al..."
3,5,I use them as shower caps & conditioning caps....
4,5,This is my go-to daily sunblock. It leaves no ...


We filter out 20000 reviews from each of the 3 categories and combine them to form a new dataframe. 

In [6]:
df_1 = new_df.query("star_rating == 1 | star_rating == 2").sample(n=20000, random_state = 2)
df_1['star_rating'] = 1
df_1.head()

,star_rating,review_body
4735045,1,"Product did absolutely nothing. No results, no..."
4576764,1,I purchased the suave 30day keratin smoothing ...
3688592,1,It's not that I hate it - I just can't use it....
4931292,1,When I received the Razorba the first thing I ...
3690743,1,I bought it in place of the obagi elastiderm e...


In [7]:
df_2 = new_df.query("star_rating == 3").sample(n=20000, random_state = 2)
df_2['star_rating'] = 2
df_2.head()

,star_rating,review_body
3063047,2,I did something I've never done in the past --...
3482781,2,This is a nice silky and shiny wig that's goo...
385503,2,Got exposed to this soap during a trip to the ...
2317014,2,Didn't realize you're supposed to use the file...
5072769,2,"I hate polishing my nails, because at least on..."


In [8]:
df_3 = new_df.query("star_rating == 4 | star_rating == 5").sample(n=20000, random_state = 2)
df_3['star_rating'] = 3
df_3.head()

,star_rating,review_body
4901168,3,The product is wonderful but I'm not sure the ...
4983887,3,Have had several shavers including Remington a...
4349368,3,The illuminted side is fantastic. As we get o...
2879398,3,I hate that Loreal stopped making this product...
1268806,3,Smells great


In [9]:
dfs = [df_1, df_2, df_3]
final_df = pd.concat(dfs)

df = shuffle(final_df, random_state = 2)
df.head()

,star_rating,review_body
3083304,1,Product is so cheaply made...wasted $$$ Worst...
2774319,1,"works ok to start out with, but start shopping..."
4959701,1,"Used this oil, didn't work, still have the toe..."
3840155,2,"It hides cleavage, but looks like a clip. Wou..."
5014932,3,I used Hawaiian Tropic Ozone Sunblock SPF70 in...


In [10]:
df['star_rating'] = df['star_rating'].astype('int')
df.head()

,star_rating,review_body
3083304,1,Product is so cheaply made...wasted $$$ Worst...
2774319,1,"works ok to start out with, but start shopping..."
4959701,1,"Used this oil, didn't work, still have the toe..."
3840155,2,"It hides cleavage, but looks like a clip. Wou..."
5014932,3,I used Hawaiian Tropic Ozone Sunblock SPF70 in...


In [11]:
df.dtypes

star_rating     int64
review_body    object
dtype: object

Handle null values in the dataframe by replacing null with a blank string. 

In [12]:
for column in ['star_rating', 'review_body']:
  print(column + " - "  + str(df[column].isnull().sum()))

star_rating - 0
review_body - 7


### Perform Data Cleaning

We perform the following actions on the dataframe: \
1. Convert to lowercase
2. Remove html and url 
3. Remove non-alphabetic characters
4. Remove extra spaces
5. Replace contractions with their original words

In [13]:
#convert to lowercase
df = df.fillna("")
df['review_body'] = df['review_body'].apply(str.lower)
df.head()

,star_rating,review_body
3083304,1,product is so cheaply made...wasted $$$ worst...
2774319,1,"works ok to start out with, but start shopping..."
4959701,1,"used this oil, didn't work, still have the toe..."
3840155,2,"it hides cleavage, but looks like a clip. wou..."
5014932,3,i used hawaiian tropic ozone sunblock spf70 in...


In [14]:
# remove html and urls
def html_url(x):
  x = re.sub('http\S+|www.\S+', '', x)
  x = re.sub('<.*?>','',x)
  return x

df['review_body'] = [html_url(x) for x in df['review_body']]
df.head()

,star_rating,review_body
3083304,1,product is so cheaply made...wasted $$$ worst...
2774319,1,"works ok to start out with, but start shopping..."
4959701,1,"used this oil, didn't work, still have the toe..."
3840155,2,"it hides cleavage, but looks like a clip. wou..."
5014932,3,i used hawaiian tropic ozone sunblock spf70 in...


In [15]:
#remove non-alphabetic
def alpha(x):
  return re.sub(r'[^a-zA-Z\s]+', '', str(x))

df['review_body'] = [alpha(x) for x in df['review_body']]
df.head()

,star_rating,review_body
3083304,1,product is so cheaply madewasted worst produ...
2774319,1,works ok to start out with but start shopping ...
4959701,1,used this oil didnt work still have the toenai...
3840155,2,it hides cleavage but looks like a clip would...
5014932,3,i used hawaiian tropic ozone sunblock spf in m...


In [16]:
def spaces(x):
  result = " ".join(x.split())
  return result

df['review_body'] = [spaces(x) for x in df['review_body']]
df.head()

,star_rating,review_body
3083304,1,product is so cheaply madewasted worst product...
2774319,1,works ok to start out with but start shopping ...
4959701,1,used this oil didnt work still have the toenai...
3840155,2,it hides cleavage but looks like a clip would ...
5014932,3,i used hawaiian tropic ozone sunblock spf in m...


In [17]:
df.head()

,star_rating,review_body
3083304,1,product is so cheaply madewasted worst product...
2774319,1,works ok to start out with but start shopping ...
4959701,1,used this oil didnt work still have the toenai...
3840155,2,it hides cleavage but looks like a clip would ...
5014932,3,i used hawaiian tropic ozone sunblock spf in m...


In [18]:
!pip install contractions

In [19]:
import contractions
def con(x):
  return contractions.fix(x)

df['review_body'] = [con(x) for x in df['review_body']]
df.head()

,star_rating,review_body
3083304,1,product is so cheaply madewasted worst product...
2774319,1,works ok to start out with but start shopping ...
4959701,1,used this oil did not work still have the toen...
3840155,2,it hides cleavage but looks like a clip would ...
5014932,3,i used hawaiian tropic ozone sunblock spf in m...


### Perform Data Preprocessing

Use WordNetLemmatizer to lemmatize all words in the corpus. 

In [20]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemm(x):
  return " ".join([lemmatizer.lemmatize(word) for word in x.split()])

df['review_body'] = [lemm(x) for x in df['review_body']]
df.head()

,star_rating,review_body
3083304,1,product is so cheaply madewasted worst product...
2774319,1,work ok to start out with but start shopping f...
4959701,1,used this oil did not work still have the toen...
3840155,2,it hide cleavage but look like a clip would be...
5014932,3,i used hawaiian tropic ozone sunblock spf in m...


## 2. Word Embeddings

<b> Load the pretrained model by google 'word2vec-google-news-300' and explore it by calculating word similarities. </b> 

In [21]:
import gensim.downloader as gensim
word2vec = gensim.load('word2vec-google-news-300')

<b> Compare semantic similarities of 3 examples </b> 

In [22]:
test_pairs = [
    ('computer', 'technology'),
    ('king', 'queen'),
    ('air', 'cold')
]

for word1, word2 in test_pairs:
  print("( " + word1 + ", " + word2 + " ) = " + str(word2vec.similarity(word1, word2)))

( computer, technology ) = 0.35386845
( king, queen ) = 0.6510956
( air, cold ) = 0.23119448


<b> Create our own Word2Vec model on the words in our corpus and check for semantic similarities of the same 3 paris as above. </b> 

In [23]:
from gensim.utils import tokenize
from nltk import word_tokenize

reviews_text = [word_tokenize(sentence) for sentence in df['review_body'].apply(str)]

In [24]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = reviews_text, vector_size = 300, window=13, min_count=9)

model.save('word2vecmodel')

In [25]:
model = Word2Vec.load('word2vecmodel')

In [26]:
for word1, word2 in test_pairs:
  print("( " + word1 + ", " + word2 + " ) = " + str(model.wv.similarity(word1, word2)))

( computer, technology ) = 0.2567897
( king, queen ) = 0.38569945
( air, cold ) = 0.63204056


## Questions

### What do you conclude from comparing vectors generated by yourself and the pretrained model? 

Conclusion: The vectors generted by our own model are less accurate as compared to the ones by the pretrained model. This is because the pretrained model has a corpus of all the words in the english language, while our model has only been trained on only the words in our dataset. \
The vectors that are generated by our own Word2Vec model are curated towards the dataset as it is purely trained upon it. The vectors used by the pretrained Google Word2Vec model provide vectors and therefore meanings in a more general domain.



### Which of the Word2Vec models seems to encode semantic similarities between words better?

The google pretrained model seems to encode semantic similarities better.

### Train Test Split

In [27]:
reviews = df['review_body']
label = df['star_rating']

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reviews, label, test_size = 0.2, random_state = 2)
print(len(X_train), len(X_test), len(y_train), len(y_test))

48000 12000 48000 12000


## 3. Simple Models

<b> Generate word embeddings and find the average value for each sentence based on Word2Vec and TfIdf. </b>

In [29]:
def word_embeddings(review, mod):
    doc = np.zeros(300)
    n = 0
    i = 0
    for i in range(len(review)):
      if review[i] in mod:
        #print(review[i])
        doc += mod[review[i]]
        n += 1
      
    if n > 0:
      return doc/n
    else:
      return doc
    
Xtrain_w2v = []
Xtest_w2v = []


In [30]:
X_train = np.array(X_train)
for rev in X_train:
    Xtrain_w2v.append(word_embeddings(rev.split(), word2vec))
    
for rev in X_test:
    Xtest_w2v.append(word_embeddings(rev.split(), word2vec))

print(len(Xtrain_w2v), len(Xtest_w2v))

48000 12000


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
Xtrain_tfidf = vectorizer.fit_transform(X_train)
Xtest_tfidf = vectorizer.transform(X_test)

print(Xtrain_tfidf.shape, Xtest_tfidf.shape)

(48000, 541283) (12000, 541283)


In [32]:
ytrain = y_train.values
ytest = y_test.values

print(len(ytrain), len(ytest))

48000 12000


### Perceptron Model

<b> Now train a perceptron model for both, the word2vec features and the tfidf features. </b> 

In [33]:
from sklearn.linear_model import Perceptron

perceptron_tfidf = Perceptron(random_state = 42)
perceptron_tfidf.fit(Xtrain_tfidf, ytrain)

perceptron_w2v = Perceptron(random_state = 42)
perceptron_w2v.fit(Xtrain_w2v, ytrain)

Perceptron(random_state=42)

In [34]:
# make predictions
ytest_tfidf_pred_perceptron = perceptron_tfidf.predict(Xtest_tfidf)
ytest_w2v_pred_perceptron = perceptron_w2v.predict(Xtest_w2v)

<b> Based on the models created, we can make predictions for the testing-data and print the accuracies. </b>

In [35]:
from sklearn.metrics import accuracy_score
print("Perceptron Model:")
print("Accuracy on Tfidf: ", accuracy_score(ytest, ytest_tfidf_pred_perceptron))
print("Accuracy on Word2Vec: ", accuracy_score(ytest, ytest_w2v_pred_perceptron))

Perceptron Model:
Accuracy on Tfidf:  0.70125
Accuracy on Word2Vec:  0.6244166666666666


### SVM Model 

<b> Similarly, we train an SVM Model for both, the Word2vec features and tfidf features and print their accuracies on the testing-data. </b> 

In [36]:
from sklearn.svm import LinearSVC
svm_tfidf = LinearSVC(random_state = 2)
svm_w2v = LinearSVC(random_state= 2)

svm_tfidf.fit(Xtrain_tfidf, ytrain)
svm_w2v.fit(Xtrain_w2v, ytrain)

ytest_tfidf_pred_svm = svm_tfidf.predict(Xtest_tfidf)
ytest_w2v_pred_svm = svm_w2v.predict(Xtest_w2v)

print("Perceptron Model:")
print("Accuracy on Tfidf: ", accuracy_score(ytest, ytest_tfidf_pred_svm))
print("Accuracy on Word2Vec: ", accuracy_score(ytest, ytest_w2v_pred_svm))

Perceptron Model:
Accuracy on Tfidf:  0.7316666666666667
Accuracy on Word2Vec:  0.6636666666666666


## Questions

### What do you conclude from comparing performances for the models trained using the two different feature types (TF-IDF and your trained Word2Vec features)?

The performance of the model using TF-IDF features is found to be better as compared to the pretrained model trained using Word2Vec features for both Perceptron as well as SVM. One reason fro this can be that the word embedding method contains a much more ‘noisy’ signal compared to TF-IDF since it is a much more complex word representation and carries much more hidden information. In our case, most of that information is unnecessary and might create false patterns in our model.

## 4. Feedforward Neural Networks


### Part A: FNN by calculating mean of all Word2Vec vectors for a review

 <b> Now we use use tensorflow to generate a feedforward neural network containing 2 hidden layers (100 and 10 nodes each) and train it for our data. </b>

In [37]:
train_y = ytrain - 1
test_y = ytest - 1


In [38]:
Ytest = np.eye(3)[test_y]
Ytrain = np.eye(3)[train_y]

In [39]:
import tensorflow as tf
from tensorflow import keras

fnn_model = keras.Sequential()
fnn_model.add(keras.layers.InputLayer(input_shape = (300,)))
fnn_model.add(keras.layers.Dense(units = 100, activation = tf.nn.leaky_relu))
fnn_model.add(keras.layers.Dense(units = 10, activation = tf.nn.leaky_relu))
fnn_model.add(keras.layers.Dense(units = 3, activation = 'softmax'))

optimizer = keras.optimizers.legacy.Adam(learning_rate = 0.005)
fnn_model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

input_shape = (300,)

fnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               30100     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
 dense_2 (Dense)             (None, 3)                 33        
                                                                 
Total params: 31,143
Trainable params: 31,143
Non-trainable params: 0
_________________________________________________________________


2023-03-01 11:18:02.331141: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [40]:
Xtest_w2v = np.array(Xtest_w2v)
Xtrain_w2v = np.array(Xtrain_w2v)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = fnn_model.fit(Xtrain_w2v, 
                        Ytrain, 
                        epochs = 50, 
                        validation_data=(Xtest_w2v, Ytest), 
                        callbacks=[early_stopping])


Epoch 1/50
1500/1500 [==============================] - 3s 1ms/step - loss: 0.8245 - accuracy: 0.6221 - val_loss: 0.7747 - val_accuracy: 0.6545
Epoch 2/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.7803 - accuracy: 0.6485 - val_loss: 0.7540 - val_accuracy: 0.6674
Epoch 3/50
1500/1500 [==============================] - 1s 949us/step - loss: 0.7690 - accuracy: 0.6553 - val_loss: 0.7844 - val_accuracy: 0.6446
Epoch 4/50
1500/1500 [==============================] - 1s 935us/step - loss: 0.7567 - accuracy: 0.6603 - val_loss: 0.7483 - val_accuracy: 0.6695
Epoch 5/50
1500/1500 [==============================] - 1s 954us/step - loss: 0.7479 - accuracy: 0.6650 - val_loss: 0.7504 - val_accuracy: 0.6701
Epoch 6/50
1500/1500 [==============================] - 1s 940us/step - loss: 0.7389 - accuracy: 0.6700 - val_loss: 0.7374 - val_accuracy: 0.6748
Epoch 7/50
1500/1500 [==============================] - 1s 944us/step - loss: 0.7326 - accuracy: 0.6720 - val_loss: 0.7462 - val

In [41]:
ypred_fnn = fnn_model.predict(Xtest_w2v)

375/375 [==============================] - 0s 559us/step


<b> Generate the accuracy for testing data for word2vec model. </b>

In [42]:
from sklearn.metrics import accuracy_score
Ypred_fnn = np.argmax(ypred_fnn, axis=1)
print("Accuracy on testing data for feedforward neural network:")
print("Accuracy on Word2Vec: ", accuracy_score(test_y, Ypred_fnn))

Accuracy on testing data for feedforward neural network:
Accuracy on Word2Vec:  0.6748333333333333


### Part B: FNN by concatenating the first 10 Word2Vec vectors for each review

 <b> We train the same FNN model by changing the training data to include the first 10 vectors instead of using mean values. </b>

In [43]:
def first10(model, words):
    word = []
    count = 0
    for i in words.split():
        if i in model:
            word.append(model[i])
            count += 1
            if count ==10:
                break
    
    while count!= 10:
        word.append(np.zeros(300))
        count += 1
    
    return np.reshape(word, 3000)

In [44]:
Xtrain = np.array(X_train)
Xtrain_fnn_10 = [first10(word2vec, vec) for vec in X_train]

Xtest = np.array(X_test)
Xtest_fnn_10 = [first10(word2vec, vec) for vec in X_test]


In [45]:
ytrain = y_train.values - 1
ytest = y_test.values - 1

Ytrain = np.zeros((len(ytrain), 3))
for i, prediction in enumerate(ytrain):
    Ytrain[i, prediction] = 1

Ytest = np.zeros((len(ytest), 3))
for i, prediction in enumerate(ytest):
    Ytest[i, prediction] = 1

In [46]:
Xtrain_fnn_10 = np.array(Xtrain_fnn_10)
Xtest_fnn_10 = np.array(Xtest_fnn_10)

In [47]:
import tensorflow as tf
from tensorflow import keras

fnn10_model = keras.Sequential()
fnn10_model.add(keras.layers.InputLayer(input_shape = (3000,)))
fnn10_model.add(keras.layers.Dense(units = 100, activation = 'relu'))
fnn10_model.add(keras.layers.Dropout(0.3))
fnn10_model.add(keras.layers.Dense(units = 10, activation = 'relu'))
fnn10_model.add(keras.layers.Dropout(0.3))
fnn10_model.add(keras.layers.Dense(units = 3, activation = 'softmax'))


optimizer = keras.optimizers.legacy.Adam(learning_rate = 0.0005)
fnn10_model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

fnn10_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               300100    
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_4 (Dense)             (None, 10)                1010      
                                                                 
 dropout_1 (Dropout)         (None, 10)                0         
                                                                 
 dense_5 (Dense)             (None, 3)                 33        
                                                                 
Total params: 301,143
Trainable params: 301,143
Non-trainable params: 0
_________________________________________________________________


In [48]:
Xtest_fnn_10 = np.array(Xtest_fnn_10)
Xtrain_fnn_10 = np.array(Xtrain_fnn_10)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = fnn10_model.fit(Xtrain_fnn_10, Ytrain, epochs = 50, validation_data=(Xtest_fnn_10, Ytest), callbacks=[early_stopping])


Epoch 1/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.9815 - accuracy: 0.4983 - val_loss: 0.9091 - val_accuracy: 0.5753
Epoch 2/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.9092 - accuracy: 0.5590 - val_loss: 0.8810 - val_accuracy: 0.5848
Epoch 3/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.8752 - accuracy: 0.5800 - val_loss: 0.8805 - val_accuracy: 0.5807
Epoch 4/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.8390 - accuracy: 0.6049 - val_loss: 0.8772 - val_accuracy: 0.5877
Epoch 5/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.8064 - accuracy: 0.6256 - val_loss: 0.8844 - val_accuracy: 0.5857
Epoch 6/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.7698 - accuracy: 0.6472 - val_loss: 0.8917 - val_accuracy: 0.5846
Epoch 7/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.7317 - accuracy: 0.6695 - val_loss: 0.9135 - val_accuracy:

In [49]:
ypred_fnn_10 = fnn10_model.predict(Xtest_fnn_10)

375/375 [==============================] - 0s 684us/step


<b> Generate the accuracy for testing data for word2vec model. </b>

In [50]:
from sklearn.metrics import accuracy_score
Ypred_fnn_10 = np.argmax(ypred_fnn_10, axis=1)
print("Accuracy on testing data for feedforward neural network:")
print("Accuracy on Word2Vec: ", accuracy_score(ytest, Ypred_fnn_10))

Accuracy on testing data for feedforward neural network:
Accuracy on Word2Vec:  0.5876666666666667


## Question

### What do you conclude by comparing accuracy values you obtain with those obtained in the “’Simple Models” section?

The accuracy values of FNN for Word2Vec are slightly better than that of SVM for Word2Vec features. The accuracy for the first 10 Word2Vec vectors for feedforward neural network is around the range of Perceptron accuracy, which is slightly lower than the FNN trained on the mean values of the vector. This is because the first 10 words in a review sometimes might not have a lot of context and meaning to them; thus not being very helpful for categorization.  

## 5. Recurrent Neural Networks

<b> Now we implement 3 types of RNN, and comapre their performance on the word2vec vectors for our dataset. We are cosidering an RNN cell with the hidden state size of 20, which means we need to truncate each review to a size of 20 words, and if any review has length less than 20, we add padding of the remaining size. </b>

In [51]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [52]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

Xtrain_w2v_rnn = []
Xtest_w2v_rnn  = []

for sentence in X_train:

    sentence_vectors = []
    sent = sentence.split()

    for word in sent:
        if word in word2vec:
            word_vectors = word2vec[word]

        sentence_vectors.append(word_vectors)

    Xtrain_w2v_rnn.append(sentence_vectors)
        
        
for sentence in X_test:
    sentence_vectors = []
    sent = sentence.split()
    for word in sent:
        if word in word2vec:
            word_vectors = word2vec[word]

        sentence_vectors.append(word_vectors)

    Xtest_w2v_rnn.append(sentence_vectors)
        

In [53]:

max_review_length = 20

Xtrain_rnn = pad_sequences(Xtrain_w2v_rnn, maxlen=max_review_length, dtype='float32', padding='post', truncating='post', value=0.0)
Xtest_rnn = pad_sequences(Xtest_w2v_rnn, maxlen=max_review_length, dtype='float32', padding='post', truncating='post', value=0.0)


In [54]:
input_dimension = len(word2vec.key_to_index)

### 5.1 Simple RNN

<b> We train a simple RNN with one masking layer and one SImpleRNN layer with hidden state size of 20. </b>

In [55]:

from tensorflow import keras

rnn_model = keras.Sequential()
#rnn_model.add(keras.layers.Embedding(input_dim = input_dimension, output_dim=300, input_length = 20, trainable = False))
rnn_model.add(keras.layers.Masking())
rnn_model.add(keras.layers.SimpleRNN(units=20, activation='relu'))
rnn_model.add(keras.layers.Dense(units=3, activation='softmax'))

optimizer = keras.optimizers.legacy.Adam(learning_rate = 0.001)
rnn_model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])

#print(rnn_model.summary())


In [56]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3, restore_best_weights=True)
rnn_model.fit(Xtrain_rnn, Ytrain, epochs=50, validation_data=(Xtest_rnn, Ytest), callbacks=[early_stopping])


Train on 48000 samples, validate on 12000 samples
Epoch 1/50


2023-03-01 11:19:04.895077: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-03-01 11:19:04.942639: W tensorflow/c/c_api.cc:291] Operation '{name:'training/Adam/sequential_2/simple_rnn/simple_rnn_cell/bias/v/Assign' id:519 op device:{requested: '', assigned: ''} def:{{{node training/Adam/sequential_2/simple_rnn/simple_rnn_cell/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/sequential_2/simple_rnn/simple_rnn_cell/bias/v, training/Adam/sequential_2/simple_rnn/simple_rnn_cell/bias/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


47616/48000 [============================>.] - ETA: 0s - loss: 0.9724 - accuracy: 0.5014

2023-03-01 11:19:10.145966: W tensorflow/c/c_api.cc:291] Operation '{name:'loss/mul' id:335 op device:{requested: '', assigned: ''} def:{{{node loss/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss/mul/x, loss/output_1_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


48000/48000 [==============================] - 6s 119us/sample - loss: 0.9725 - accuracy: 0.5016 - val_loss: 0.9054 - val_accuracy: 0.5570
Epoch 2/50
48000/48000 [==============================] - 5s 109us/sample - loss: 0.8896 - accuracy: 0.5751 - val_loss: 0.8708 - val_accuracy: 0.5839
Epoch 3/50
48000/48000 [==============================] - 5s 108us/sample - loss: 0.8604 - accuracy: 0.5949 - val_loss: 0.8706 - val_accuracy: 0.5847
Epoch 4/50
48000/48000 [==============================] - 5s 108us/sample - loss: 0.8446 - accuracy: 0.6042 - val_loss: 0.8410 - val_accuracy: 0.6022
Epoch 5/50
48000/48000 [==============================] - 5s 108us/sample - loss: 0.8280 - accuracy: 0.6181 - val_loss: 0.8427 - val_accuracy: 0.6047
Epoch 6/50
48000/48000 [==============================] - 6s 116us/sample - loss: 0.8179 - accuracy: 0.6251 - val_loss: 0.8271 - val_accuracy: 0.6102
Epoch 7/50
48000/48000 [==============================] - 6s 123us/sample - loss: 0.8095 - accuracy: 0.6300 - v

In [57]:
ypred_rnn = rnn_model.predict(Xtest_rnn)

2023-03-01 11:25:03.238749: W tensorflow/c/c_api.cc:291] Operation '{name:'sequential_2/dense_6/Softmax' id:259 op device:{requested: '', assigned: ''} def:{{{node sequential_2/dense_6/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](sequential_2/dense_6/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


<b> Generate the accuracy for testing data for word2vec model. </b> 

In [58]:
from sklearn.metrics import accuracy_score
Ypred_rnn = np.argmax(ypred_rnn, axis=1)
print("Accuracy on testing data for recurrent neural network:")
print("Accuracy on Word2Vec: ", accuracy_score(ytest, Ypred_rnn))

Accuracy on testing data for recurrent neural network:
Accuracy on Word2Vec:  0.62875


## Question

### What do you conclude by comparing accuracy values you obtain with those obtained with feedforward neural network models? 

The accuracy values for FNN and RNN are very similar, and can vary a little based on hyperparameter tuning, number of epochs and activation used on each layer of the network.

### 5.2 GRU (Gated Recurrent Unit):

<b> We train a GRU model with one masking layer and one GRU layer with hidden state size of 20. </b> 

In [59]:
from tensorflow import keras

gru_model = keras.Sequential()
#rnn_model.add(keras.layers.Embedding(input_dim = input_dimension, output_dim=300, input_length = 20, trainable = False))
gru_model.add(keras.layers.Masking())
gru_model.add(keras.layers.GRU(units=20, activation='relu'))
gru_model.add(keras.layers.Dense(units=3, activation='softmax'))

optimizer = keras.optimizers.legacy.Adam(learning_rate = 0.001)
gru_model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])

#print(rnn_model.summary())

In [60]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3, restore_best_weights=True)
gru_model.fit(Xtrain_rnn, Ytrain, epochs=15 , validation_data=(Xtest_rnn, Ytest), callbacks=[early_stopping])

Train on 48000 samples, validate on 12000 samples
Epoch 1/15


2023-03-01 11:25:04.215237: W tensorflow/c/c_api.cc:291] Operation '{name:'total_1/Assign' id:969 op device:{requested: '', assigned: ''} def:{{{node total_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](total_1, total_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


47968/48000 [============================>.] - ETA: 0s - loss: 0.8931 - accuracy: 0.5702

2023-03-01 11:25:12.288229: W tensorflow/c/c_api.cc:291] Operation '{name:'loss_1/mul' id:1040 op device:{requested: '', assigned: ''} def:{{{node loss_1/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss_1/mul/x, loss_1/output_1_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


48000/48000 [==============================] - 9s 182us/sample - loss: 0.8930 - accuracy: 0.5703 - val_loss: 0.8244 - val_accuracy: 0.6221
Epoch 2/15
48000/48000 [==============================] - 8s 172us/sample - loss: 0.8063 - accuracy: 0.6306 - val_loss: 0.7936 - val_accuracy: 0.6368
Epoch 3/15
48000/48000 [==============================] - 8s 173us/sample - loss: 0.7805 - accuracy: 0.6458 - val_loss: 0.7714 - val_accuracy: 0.6528
Epoch 4/15
48000/48000 [==============================] - 8s 172us/sample - loss: 0.7602 - accuracy: 0.6554 - val_loss: 0.7632 - val_accuracy: 0.6567
Epoch 5/15
48000/48000 [==============================] - 8s 174us/sample - loss: 0.7454 - accuracy: 0.6624 - val_loss: 0.7556 - val_accuracy: 0.6600
Epoch 6/15
48000/48000 [==============================] - 9s 180us/sample - loss: 0.7320 - accuracy: 0.6713 - val_loss: 0.7502 - val_accuracy: 0.6626
Epoch 7/15
48000/48000 [==============================] - 9s 185us/sample - loss: 0.7207 - accuracy: 0.6762 - v

In [61]:
ypred_gru = gru_model.predict(Xtest_rnn)

2023-03-01 11:27:15.492140: W tensorflow/c/c_api.cc:291] Operation '{name:'sequential_3/dense_7/Softmax' id:964 op device:{requested: '', assigned: ''} def:{{{node sequential_3/dense_7/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](sequential_3/dense_7/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


<b> Generate the accuracy for testing data for word2vec model. </b> 

In [62]:
from sklearn.metrics import accuracy_score
Ypred_gru = np.argmax(ypred_gru, axis=1)
print("Accuracy on testing data for GRU neural network:")
print("Accuracy on Word2Vec: ", accuracy_score(ytest, Ypred_gru))

Accuracy on testing data for GRU neural network:
Accuracy on Word2Vec:  0.6578333333333334


### 5.3 LSTM network:

<b> We train an LSTM model with one masking layer and one LSTM layer with hidden state size of 20. </b>

In [63]:
from tensorflow import keras

lstm_model = keras.Sequential()
#rnn_model.add(keras.layers.Embedding(input_dim = input_dimension, output_dim=300, input_length = 20, trainable = False))
lstm_model.add(keras.layers.Masking())
lstm_model.add(keras.layers.LSTM(units=20, activation='relu'))
lstm_model.add(keras.layers.Dense(units=3, activation='softmax'))

optimizer = keras.optimizers.legacy.Adam(learning_rate = 0.001)
lstm_model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])

#print(rnn_model.summary())

In [64]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3, restore_best_weights=True)
lstm_model.fit(Xtrain_rnn, Ytrain, epochs=20, validation_data=(Xtest_rnn, Ytest), callbacks=[early_stopping])

Train on 48000 samples, validate on 12000 samples
Epoch 1/20


2023-03-01 11:27:16.724986: W tensorflow/c/c_api.cc:291] Operation '{name:'training_4/Adam/sequential_4/dense_8/kernel/v/Assign' id:1981 op device:{requested: '', assigned: ''} def:{{{node training_4/Adam/sequential_4/dense_8/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training_4/Adam/sequential_4/dense_8/kernel/v, training_4/Adam/sequential_4/dense_8/kernel/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


47776/48000 [============================>.] - ETA: 0s - loss: 0.8968 - accuracy: 0.5742

2023-03-01 11:27:26.608582: W tensorflow/c/c_api.cc:291] Operation '{name:'loss_2/mul' id:1761 op device:{requested: '', assigned: ''} def:{{{node loss_2/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss_2/mul/x, loss_2/output_1_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


48000/48000 [==============================] - 11s 225us/sample - loss: 0.8965 - accuracy: 0.5742 - val_loss: 0.8263 - val_accuracy: 0.6217
Epoch 2/20
48000/48000 [==============================] - 10s 208us/sample - loss: 0.8060 - accuracy: 0.6300 - val_loss: 0.8013 - val_accuracy: 0.6342
Epoch 3/20
48000/48000 [==============================] - 11s 236us/sample - loss: 0.7748 - accuracy: 0.6459 - val_loss: 0.7913 - val_accuracy: 0.6366
Epoch 4/20
48000/48000 [==============================] - 10s 204us/sample - loss: 0.7541 - accuracy: 0.6569 - val_loss: 0.7710 - val_accuracy: 0.6528
Epoch 5/20
48000/48000 [==============================] - 10s 199us/sample - loss: 0.7365 - accuracy: 0.6677 - val_loss: 0.7660 - val_accuracy: 0.6520
Epoch 6/20
48000/48000 [==============================] - 11s 224us/sample - loss: 0.7216 - accuracy: 0.6758 - val_loss: 0.7690 - val_accuracy: 0.6498
Epoch 7/20
48000/48000 [==============================] - 10s 209us/sample - loss: 0.7087 - accuracy: 0.6

In [65]:
ypred_lstm = lstm_model.predict(Xtest_rnn)

2023-03-01 11:31:57.632217: W tensorflow/c/c_api.cc:291] Operation '{name:'sequential_4/dense_8/Softmax' id:1685 op device:{requested: '', assigned: ''} def:{{{node sequential_4/dense_8/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](sequential_4/dense_8/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


<b> Generate the accuracy for testing data for word2vec model. </b>

In [66]:
from sklearn.metrics import accuracy_score
Ypred_lstm = np.argmax(ypred_lstm, axis=1)
print("Accuracy on testing data for LSTM neural network:")
print("Accuracy on Word2Vec: ", accuracy_score(ytest, Ypred_lstm))

Accuracy on testing data for LSTM neural network:
Accuracy on Word2Vec:  0.6485833333333333


## Question

### What do you conclude by comparing accuracy values you obtain by GRU, LSTM, and simple RNN?

GRU and LSTM networks perform well and give very similar accuracies on the testing data. SimpleRNN model does not perform as well because it has a short-term memory problem. 

In [67]:
# Checking Package versions for all libraries

# import sys
# import sklearn as sk
# import gensim as gen
# print("Python" , sys.version)
# print("Pandas" , pd.__version__)
# print("Numpy " , np.version.version)
# print("Scikit-learn" , sk.__version__)
# print("NLTK ", nltk.__version__)
# print("Tensorflow" , tf.__version__)
# print("Gensim " , gen.__version__)